In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
device

'cuda'

In [3]:
data_path = '../data/tiny_shakespeare.txt'
eval_iters = 200

In [4]:
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
VOCAB_SIZE = len(chars)
n_embd = 32
print(''.join(chars))
print(VOCAB_SIZE)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda v: ''.join([itos[i] for i in v])

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
BLOCK_SIZE = 8

In [9]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    #print(f'When input is {context} the target is: {target}')

In [15]:
torch.manual_seed(1337)
BATCH_SIZE = 32
BLOCK_SIZE = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')

print('----')

for b in range(BATCH_SIZE):
    for t in range(BLOCK_SIZE):
        context = xb[b, :t+1]
        target = yb[b, t]
        #print(f'When input is {context.tolist()} the target is: {target}')

----


In [16]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [18]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(VOCAB_SIZE, n_embd)
        self.position_embedding_table = nn.Embedding(256, n_embd)
        self.lm_head = nn.Linear(n_embd, VOCAB_SIZE)
    
    def forward(self, idx, targets=None):
        b, t = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(t, device=device))
        x = tok_emb + pos_emb
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            b, t, c = logits.shape        
            logits = logits.view(b*t, c)
            targets = targets.view(b*t)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx = idx.to(device)
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)
print(logits.shape, loss)
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([256, 65]) tensor(4.4348, device='cuda:0', grad_fn=<NllLossBackward>)

WFwwcHPmR.x,fDs$aAzxzn$NuX
YoR&$LMtof ,ENvBb3BV!OW;Kl!lL!,$e3 irwe-ERngciK;ExPiHFGE?rov EsSXUB;qWk J


In [19]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
batch_size = 32
epochs = 10000

In [20]:
for epoch in range(epochs):
    if epoch % 1000 == 0:
        losses = estimate_loss()
        print(f'step {epoch}: train loss {losses["train"]:.4f}, val loss {losses["val"]:.4f}')
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0: train loss 4.5080, val loss 4.5095
step 1000: train loss 2.5873, val loss 2.6069


KeyboardInterrupt: 

In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=300)[0].tolist()))

## Attention Mechanism

In [ ]:
torch.manual_seed(1337)
b, t, c = 4, 8, 2
x = torch.randn(b, t, c)
x.shape

In [ ]:
xbow = torch.zeros((b,t,c))
for i in range(b):
    for j in range(t):
            xprev = x[i, :j+1]
            xbow[i,j] = torch.mean(xprev, 0)

In [ ]:
wei = torch.tril(torch.ones(t, t))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
torch.allclose(xbow, xbow2)

In [ ]:
tril = torch.tril(torch.ones(t, t))
wei = torch.zeros((t, t))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)